# Overview
Load the movie ratings data (as in the HW3-recommender-system) and use matrix factorization technique(s) and predict the missing ratings from the test data. Measure the RMSE.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed

# Input data files are available in the read-only "./data/" directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Output fils will be in ./output directory

./data/users.csv
./data/test.csv
./data/train.csv
./data/movies.csv


In [109]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn.model_selection import train_test_split
from scipy.sparse import coo_matrix, csr_matrix
from scipy.spatial.distance import jaccard, cosine 
# from pytest import approx

from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF

**1- Let's load the data**

In [104]:
# Load data
train_ratings = pd.read_csv('./data/train.csv')
test_ratings = pd.read_csv('./data/test.csv')
users = pd.read_csv('./data/users.csv')
movies = pd.read_csv('./data/movies.csv')

print(train_ratings)
print(test_ratings)
# print(users.info())
# print(movies.head())


         uID   mID  rating
0        744  1210       5
1       3040  1584       4
2       1451  1293       5
3       5455  3176       2
4       2507  3074       5
...      ...   ...     ...
700141  1184  2916       3
700142   137  1372       5
700143   195  2514       3
700144  1676  2566       3
700145  4611  1888       1

[700146 rows x 3 columns]
         uID   mID  rating
0       2233   440       4
1       4274   587       5
2       2498   454       3
3       2868  2336       5
4       1636  2686       5
...      ...   ...     ...
300058   810   247       4
300059  1193  3210       4
300060  6039  2289       4
300061  5397   429       3
300062  1912   117       4

[300063 rows x 3 columns]


**2- Let's create train and test dataframe such that we have users along rows and movies along columns and fill correspinding rating for uID-mID. We fill 0 where there is no rating**

In [105]:
# Assuming you already have train_ratings, test_ratings, users, and movies dataframes

# Create a pivot table for train data
traindf = train_ratings.pivot(index='uID', columns='mID', values='rating')
# print(traindf)

# Create a pivot table for test data
testdf = test_ratings.pivot(index='uID', columns='mID', values='rating')
# print(testdf)

# Merge users and movies data to ensure all users and movies are included in the final dataframes
users.set_index('uID', inplace=True)
movies.set_index('mID', inplace=True)

# Add missing users and movies in traindf and testdf with 0 values
traindf = traindf.reindex(index=users.index, columns=movies.index, fill_value=0)
testdf = testdf.reindex(index=users.index, columns=movies.index, fill_value=0)

# Reset index for both dataframes
traindf = traindf.reset_index()
testdf = testdf.reset_index()

# Reset column names
traindf.columns.name = None
testdf.columns.name = None

# Fill missing values with 0
traindf = traindf.fillna(0)
testdf = testdf.fillna(0)
print(traindf.shape)
print(testdf.shape)







(6040, 3884)
(6040, 3884)


**3- We have a uId column in out dataframes lets drop that**

In [106]:
# print(traindf)
# print(testdf)
# Drop the 'uID' column from traindf and testdf
traindf = traindf.drop('uID', axis=1)
testdf = testdf.drop('uID', axis=1)

# Print the modified DataFrames without the 'uID' column
# print(traindf)
print(testdf)

traindf.columns = traindf.columns.astype(str)
testdf.columns = testdf.columns.astype(str)

      1     2     3     4     5     6     7     8     9     10    ...  3943  \
0      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
1      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
2      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
3      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
4      0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
...    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
6035   0.0   0.0   0.0   0.0   0.0   3.0   0.0   0.0   0.0   0.0  ...   0.0   
6036   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6037   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6038   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
6039   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   

      3944  3945  3946  3947  3948  3949  3950  395

**4- Let's build our non-matrix factorization model using traindf and then we will use inverse-transform to get predicted ratings for all movies. Then we will use that to compare againt testdf to asses performance using rmse**

We have chosen 3 components because we have 18 geners in the movies so we expect matrix factorization to happen according to that. But usually 1 movie has around 2-3 genres.

In [117]:
# Apply matrix factorization using Truncated SVD
n_components = 3
svd = TruncatedSVD(n_components=n_components)
user_movie_matrix_reduced = svd.fit_transform(traindf)

# Predict missing ratings for the test data with the reduced matrix
predicted_ratings = svd.inverse_transform(user_movie_matrix_reduced)
# print(predicted_ratings)

# Convert predicted_ratings NumPy array to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=traindf.index, columns=traindf.columns)

# Extract predicted ratings for the test data using the same user-movie combinations
predicted_test_ratings_df = predicted_ratings_df[testdf != 0]
predicted_test_ratings_df = predicted_test_ratings_df.fillna(0)

# Print the DataFrame of predicted test ratings
print(predicted_test_ratings_df)

# Calculate RMSE
rmse = sqrt(mean_squared_error(testdf, predicted_test_ratings_df))

print("RMSE:", rmse)


        1    2    3    4    5         6    7    8    9   10  ...  3943  3944  \
0     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...   ...  ...  ...  ...  ...       ...  ...  ...  ...  ...  ...   ...   ...   
6035  0.0  0.0  0.0  0.0  0.0  1.414435  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6036  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6037  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6038  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6039  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   

      3945  3946  3947  3948  3949  395

In [118]:
# Apply matrix factorization using NMF
nmf = NMF(n_components = 3, random_state = 42, init="nndsvda", solver="mu", beta_loss="kullback-leibler", max_iter=1000)
user_movie_matrix_reduced = nmf.fit_transform(traindf)

# Predict missing ratings for the test data with the reduced matrix
predicted_ratings = nmf.inverse_transform(user_movie_matrix_reduced)
# print(predicted_ratings)

# Convert predicted_ratings NumPy array to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=traindf.index, columns=traindf.columns)

# Extract predicted ratings for the test data using the same user-movie combinations
predicted_test_ratings_df = predicted_ratings_df[testdf != 0]
predicted_test_ratings_df = predicted_test_ratings_df.fillna(0)

# Print the DataFrame of predicted test ratings
print(predicted_test_ratings_df)

# Calculate RMSE
rmse = sqrt(mean_squared_error(testdf, predicted_test_ratings_df))

print("RMSE:", rmse)

        1    2    3    4    5         6    7    8    9   10  ...  3943  3944  \
0     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
1     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
2     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
3     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
4     0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
...   ...  ...  ...  ...  ...       ...  ...  ...  ...  ...  ...   ...   ...   
6035  0.0  0.0  0.0  0.0  0.0  1.580041  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6036  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6037  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6038  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   
6039  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  ...   0.0   0.0   

      3945  3946  3947  3948  3949  395

# Discuss the results and why sklearn's non-negative matrix facorization library did not work well compared to simple baseline or similarity-based methods we’ve done in Module 3. Can you suggest a way(s) to fix it? 

Ans- We can see the non-negative matrix facorization rmse is very different from from the simple baseline or similarity-based methods. It could be because we have a sparse matrix with lot of missing ratings for user-movie pairs and we fill them with 0. Since there are lot of 0 values it throws off the loss function in NMF. If we could use only non-zero values in NMF calculation and rmse then it could improve.

# Github Link

https://github.com/arunattri26/MSDS/blob/main/DTSA_5510_Unsupervised_Learning/HW4/Movie/MovieRatings.ipynb